In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
pwd

'c:\\Users\\shrin\\Documents\\Python_Scripts\\adept\\Job-Classification'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig():
    root_dir: Path
    data_path: Path

In [5]:
from jobClassification.constants import *
from jobClassification.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )

        return data_transformation_config

In [25]:
import os
from jobClassification.logging import logger
from sentence_transformers import InputExample, evaluation
from datasets import load_dataset, load_from_disk, DatasetDict, Dataset
import torch
from torch.utils.data import DataLoader

import pandas as pd
from jobClassification.utils.common import get_clean_job_str
from sklearn.model_selection import train_test_split


In [31]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config


    def get_dataloader(self, data, split):
        examples = []
        data = data[split]
        n_examples = data.num_rows

        for i in range(n_examples):
            example = data[i]
            examples.append(InputExample(texts=[example['CLEAN_JOB'], example['ONET_NAME']], label=float(1)))
        logger.info(f"in {split}, We have a {type(examples)} of length {len(examples)} containing {type(examples[0])}'s.")
        dataloader = DataLoader(examples, shuffle=True, batch_size=16)
        return examples, dataloader

    def convert(self):

        # get csv data to df
        train_df = pd.read_csv(self.config.data_path+"raw_train/train_data.csv").head()
        print("TITLE_RAW"  in train_df.columns)
        test_df = pd.read_csv(self.config.data_path+"raw_test/test_data.csv").head()

        # add clean col to df
        train_df["CLEAN_JOB"] = train_df.apply(lambda x:get_clean_job_str(x["TITLE_RAW"], x["BODY"]), axis=1)
        test_df["CLEAN_JOB"] = test_df.apply(lambda x:get_clean_job_str(x["TITLE_RAW"], x["BODY"]), axis=1)

        # split train/val/test data
        train_ratio = 0.85
        val_ratio = 0.15
        train_df, val_df = train_test_split(train_df, test_size=1 - train_ratio, random_state=42, shuffle=True)

        final_data = DatasetDict({
            "train" : Dataset.from_pandas(train_df).remove_columns(["__index_level_0__"]),
            "val" : Dataset.from_pandas(val_df).remove_columns(["__index_level_0__"]),
            "test" : Dataset.from_pandas(test_df)
        })
        #final_data.save_to_disk( os.path.join(self.config.data_path,"final_data/"))

        dataset = final_data#load_from_disk( os.path.join(self.config.data_path,"final_data/"))

        
        train_examples, train_dataloader = self.get_dataloader(dataset, "train")
        torch.save(train_dataloader, os.path.join(self.config.root_dir,"train.pth"))

        
        val_examples, val_dataloader = self.get_dataloader(dataset, "val")
        val_evaluator = evaluation.EmbeddingSimilarityEvaluator([],[],[]).from_input_examples(examples=val_examples)
        torch.save(val_dataloader, os.path.join(self.config.root_dir,"val.pth"))
        torch.save(val_evaluator, os.path.join(self.config.root_dir,"val_eval.pth"))

        test_examples, test_dataloader = self.get_dataloader(dataset, "test")
        test_evaluator = evaluation.EmbeddingSimilarityEvaluator([],[],[]).from_input_examples(examples=test_examples)
        torch.save(test_dataloader, os.path.join(self.config.root_dir,"test.pth"))
        torch.save(test_evaluator, os.path.join(self.config.root_dir,"test_eval.pth"))


In [32]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2023-11-27 02:18:20,711: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-27 02:18:20,714: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-27 02:18:20,716: INFO: common: created directory at: artifacts]
[2023-11-27 02:18:20,718: INFO: common: created directory at: artifacts/data_transformation]
True
[2023-11-27 02:18:22,078: INFO: 1858167745: in train, We have a <class 'list'> of length 4 containing <class 'sentence_transformers.readers.InputExample.InputExample'>'s.]
[2023-11-27 02:18:22,086: INFO: 1858167745: in val, We have a <class 'list'> of length 1 containing <class 'sentence_transformers.readers.InputExample.InputExample'>'s.]
[2023-11-27 02:18:22,091: INFO: 1858167745: in test, We have a <class 'list'> of length 5 containing <class 'sentence_transformers.readers.InputExample.InputExample'>'s.]
